# First Iteration Code Development and Testing

## Algorithmic Trading - Applying Moving Average

### Importing Libraries and Dependencies

In [56]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import warnings
warnings.simplefilter(action="ignore")

In [57]:
# Input Parameters

# Window runs: Short Term (9-21, 21-50, 9-50)
# Long Term: 50-100, 21-100, Golden Cross - Death Cross: 50-200
# Objective: Which of the above combinations gives the best return for each stock

# Set the short window and long windows
short_window = 50
long_window = 200

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

### Importing Source CSV as Dataframe

In [88]:
# Reading CSV
# Set the file path
filepath = Path("resources/MRNA.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
mrna_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
mrna_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-07,17.540001,17.940001,16.180000,16.270000,16.270000,1585400
1,2019-01-08,17.350000,17.468000,16.049999,16.950001,16.950001,1892700
2,2019-01-09,17.200001,17.250000,16.150000,16.330000,16.330000,1187400
3,2019-01-10,16.360001,16.510000,16.000000,16.480000,16.480000,487300
4,2019-01-11,16.600000,17.059999,16.530001,17.000000,17.000000,580600


### Generating Moving Average Crossover Signal

In [89]:
# Grab just the `date` and `close` from the dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA200"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA200"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA200,Signal,Entry/Exit
Date,,,,,
2019-01-07,16.270000,NaN,NaN,0.0,NaN
2019-01-08,16.950001,NaN,NaN,0.0,0.0
2019-01-09,16.330000,NaN,NaN,0.0,0.0
2019-01-10,16.480000,NaN,NaN,0.0,0.0
2019-01-11,17.000000,NaN,NaN,0.0,0.0
2019-01-14,16.740000,NaN,NaN,0.0,0.0
2019-01-15,16.590000,NaN,NaN,0.0,0.0
2019-01-16,16.260000,NaN,NaN,0.0,0.0
2019-01-17,16.309999,NaN,NaN,0.0,0.0


 ### Plotting Entry/Exit Points of Moving Average Crossover Trading Strategy

In [90]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA200']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [107]:
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                 :Curve   [Date]   (Close)
   .NdOverlay.I             :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Portfolio_Total :Scatter   [Date]   (Portfolio Total)

### Backtest the Trading Strategy

In [92]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["Close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,Close,SMA50,SMA200,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2020-12-29,114.389999,107.586200,72.428575,1.0,0.0,500.0,0.0,57194.9995,90125.0,147319.9995,0.010252,0.47320
2020-12-30,111.129997,108.389599,72.843325,1.0,0.0,500.0,0.0,55564.9985,90125.0,145689.9985,-0.011064,0.45690
2020-12-31,104.470001,109.052800,73.207775,1.0,0.0,500.0,0.0,52235.0005,90125.0,142360.0005,-0.022857,0.42360
2021-01-04,111.730003,109.920000,73.625075,1.0,0.0,500.0,0.0,55865.0015,90125.0,145990.0015,0.025499,0.45990
2021-01-05,109.180000,110.686800,74.029975,1.0,0.0,500.0,0.0,54590.0000,90125.0,144715.0000,-0.008733,0.44715


### Plot Entry/Exit Points of Backtest Results

In [93]:
# Visualize exit position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                 :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [94]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [95]:
### Calculate and Assign Portfolio Evaluation Metrics

In [96]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']]
sortino_ratio_df.loc[:,'Downside Returns'] = 0
sortino_ratio_df.head()

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.201902
Cumulative Returns,0.44715
Annual Volatility,0.181541
Sharpe Ratio,1.11216
Sortino Ratio,1.83617


### Prepare Trade Evaluation Metrics DataFrame

In [97]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [98]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [99]:
### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [100]:
price_df = signals_df[['Close', 'SMA50', 'SMA200']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [101]:
### Create Hvplot Table of Portfolio Metrics

In [102]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [103]:
### Create Hvplot Table of Trade Metrics

In [104]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]